# Custom pyiron jobs

While there are a variety of `pyiron_*` libraries provided and supported by the pyiron organization, you may not find exactly what you're looking for and might want to make custom extensions to pyiron.
Here, we will show how to create new pyiron jobs that perform custom calculations.

As with other times we use pyiron, we'll import a project and -- for this demo -- make sure that the working directory is totally clean:

In [1]:
from pyiron_base import Project
pr = Project('tmp')
pr.remove_jobs(silently=True, recursive=True)

  0%|          | 0/1 [00:00<?, ?it/s]

## Python-only jobs

The simplest type of new job is one that runs python code. These can inherit from the `PythonTemplateJob`.

While all user-facing tools should be accessible directly from the `Project` instance, some developer tools need to be imported from their respective modules:

In [2]:
from pyiron_base import PythonTemplateJob

In [3]:
class HelloWorldJob(PythonTemplateJob):
    
    def __init__(self, project, job_name):
        super().__init__(project, job_name) 
        # Initial values can be added to input and output fields
        # This is advisable, as it makes the expected IO tab-completable
        self.input.num_exclamations = 1
        
        self.output.message = None
        
    # This is the function executed during normal operation (on `.run()`)
    def run_static(self):
        # It's good form to set the status to "running" when you start your calculation
        self.status.running = True
        
        # Do whatever computing you want and populate the output
        self.output.message = "Hello world" + "!" * self.input.num_exclamations
        
        # After computing, we need to manually invoke `to_hdf` to serialize the data
        self.to_hdf()
        # Everything in the `input` and `output` fields will get written to file
        
        # Don't forget to update the job status when you are finished
        self.status.finished = True
        # Normally, you will want to set the status to `finished`, but other 
        # choices include:
        #     initialized, appended, created, submitted, running, aborted, 
        #     collect, suspended, refresh, and busy
        # E.g., for a python-only job it may make sense to set the status 
        # to `aborted` if a try/except clause runs to the exception.

In [4]:
job = pr.create_job(job_type=HelloWorldJob, job_name="hello")
job.run()

The job hello was saved and received the ID: 332


We can now access the output directly on our job instance:

In [5]:
job.output.message

'Hello world!'

And we can also see it serialized in the job's HDF file:

In [6]:
job['storage/output']

DataContainer({'message': HDFStub({'groups': [], 'nodes': ['HDF_VERSION', 'NAME', 'OBJECT', 'READ_ONLY', 'TYPE', 'VERSION', 'message__index_0']}, message__index_0)})

Finally, we can de-serialize the job into a new object and see that we recover our input and output:

In [7]:
loaded = pr.load(job.name)
print(
    loaded.input.num_exclamations,
    loaded.output.message
)

1 Hello world!


## Shell-based jobs

TODO